In [27]:
pip install urielplus

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [32]:
pip install torch


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [33]:
pip install torchvision

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 32.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [39]:
pip install torchsummary

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [42]:
pip install graphviz

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [97]:
import sys, evaluator, importlib, pathlib, inspect

print("File on disk:", pathlib.Path('/Users/matteosalloum/Desktop/langrank/evaluator.py').stat().st_mtime)
print("Loaded module:", evaluator.__file__)
print("Loaded mtime :", pathlib.Path(evaluator.__file__).stat().st_mtime)

print("Has GeographicCalculator?", hasattr(evaluator, "GeographicCalculator"))

File on disk: 1754396597.585598
Loaded module: /Users/matteosalloum/Desktop/langrank/evaluator.py
Loaded mtime : 1754396597.585598
Has GeographicCalculator? True


In [98]:
import importlib, evaluator
importlib.reload(evaluator)          # Jupyter / VS Code interactive
from evaluator import GeographicCalculator

Test
Test1.25


In [99]:
import evaluator
from evaluator import DistanceCalculator, SyntacticCalculator, MorphologicalCalculator, InventoryCalculator, PhonologicalCalculator, FeaturalCalculator, ScripturalCalculator, GeographicCalculator, GenericCalculator, IslandCalculator, LangRankEvaluator
script = GenericCalculator('data/URIEL_Scriptural.csv')
islands = IslandCalculator('data/URIELPlus_Union_SoftImpute.csv')
# geo = GenericCalculator('data/URIEL_Geography.csv')
# gen = GenericCalculator('data/URIEL_Phylogeny.csv')
# typ_file = 'selection_result/imputed_laplacian_300.csv'
typ_file = 'data/URIELPlus_Union.csv'
syn = SyntacticCalculator(typ_file)
morph = MorphologicalCalculator(typ_file)
inv = InventoryCalculator(typ_file)
phon = PhonologicalCalculator(typ_file)
feat = FeaturalCalculator(typ_file)
geo = GeographicCalculator(1) #Here 0 represents the basic langauge centroid style (where two mono-country languages are thought to have 0 distance. This can be changed to 1
evaluator = LangRankEvaluator(
    calculators={
    'syntactic': syn,
    'morphological': morph,
    'inventory': inv,
    'phonological': phon,
    'featural': feat,
    'scriptural': script,
    'islands': islands,
    'geographic': geo,
    },
    iso_map_file='data/code_mapping.csv', # path to the ISO to Glottocode mapping file
)

Test1.5
Test2
Test8
Test1.5
Test2
Test1.5
Test2
Test3
Test1.5
Test2
Test7
Test1.5
Test2
Test4
Test1.5
Test2
Test5
Test1.5
Test2
Test6
Test1.5
Test2
HELLO RUNNING INTEGRATE_GEO_DATA
Test9


In [100]:
baseline_ndcg = {
    'mt': [],
    'dep': [],
    'el': [],
    'pos': [],
    'taxi1500': [],
    'xnli': []
}

In [101]:
is_baseline = True
features = ['syntactic', 'phonological', 'inventory', 'featural', 'geographic']
features += ['morphological']
features += ['scriptural']
# features += ['islands']
task_col_name = 'task_lang'
transfer_col_name = 'transfer_lang'

tasks = {
    'mt': ('BLEU', True),
    'dep': ('accuracy', True),
    'el': ('accuracy', True),
    'pos': ('accuracy', True),
    'taxi1500': ('f1_score', False),
    'xnli': ('accuracy', False),
}

for task in tasks:
    # replace/add distances in the `distance_types` columns of the dataset CSV using the corresponding calculators passed into the evaluator.
    evaluator.replace_distances(
        dataset_path=f'data/{task}.csv', # path to the task dataset (the one containing columns for task lang, transfer lang, performance)
        distance_types=features, # list of distance types to replace in the dataset. these should match the keys of the dict passed into the evaluator.
        task_col_name=task_col_name, # name of the task language column in your dataset
        transfer_col_name=transfer_col_name, # name of the transfer language column in your dataset
        iso_conversion=tasks[task][1] # indicate whether to convert lang codes in your dataset from ISO to Glottocode using the file in self.iso_map_file
    )
    # run LangRank and evaluate task performance
    score = evaluator.evaluate(
        dataset_path=f'data/{task}_updated.csv',
        features=features, # list of columns in the dataset CSV to use for evaluation
        performance_col_name=tasks[task][0], # name of the column in the dataset CSV containing task performance scores
        task_col_name=task_col_name,
        transfer_col_name=transfer_col_name,
        baseline_ndcg_scores=baseline_ndcg[task], # list of baseline NDCG scores for the task
    )
    if is_baseline:
        baseline_ndcg[task].append(score[1])
        print(f'Task: {task} NDCG: {score[0]}')
    else:
        print(f'Task: {task} NDCG: {score[0]} (p-value: {score[2]})')

Task: mt NDCG: 31.45504777195871
Task: dep NDCG: 68.43418032070102
Task: el NDCG: 68.83701170460759
Task: pos NDCG: 13.621003532640575
Task: taxi1500 NDCG: 25.218574388980436
Task: xnli NDCG: 80.37092373692253
